##### Imports


In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from importlib import reload
import sys

In [87]:
sys.path.append("..")

##### Extracting data


In [88]:
zip_path = "../../data/loc-brightkite_totalCheckins.txt.zip"
output_path = "../../data/"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(output_path)

##### Reading data


In [89]:
with open("../../data/loc-brightkite_totalCheckins.txt", "r") as file:
    lines = file.readlines()

data = [line.split() for line in lines]

df = pd.DataFrame(data)

df.columns = ["user_id", "timestamp", "latitude", "longitude", "location_id"]

##### Drop empty values of location id

We are not interested in empty values of location id since they will be the cache keys


In [90]:
df.dropna(subset=["location_id"], inplace=True)

##### Sort values by timestamp

We will preserve natural order, even between multiple users, since we will use them as tenants.


In [91]:
df.sort_values(by="timestamp", inplace=True)

##### Select 500 users with largest number of different location ids

Users with small amounts of different location ids will be easier for cache algorithm.


In [92]:
top_users = df.groupby("user_id")["location_id"].nunique().nlargest(500).index.tolist()

selected_df = df[df["user_id"].isin(top_users)].copy()

##### Convert location ids, to smaller number, to make processing easier


In [93]:
selected_df["location_id_number"] = selected_df.groupby(["user_id"])[
    "location_id"
].transform(lambda x: x.rank(method="dense").astype(int))

##### Drop columns that will not be used anymore.


In [94]:
selected_df.drop(
    columns=["timestamp", "latitude", "longitude", "location_id"], inplace=True
)

##### Cache size for baseline comparison cache (Belady Algorithm)

This will be used to select tenants with more complex sequence of locations.


In [95]:
CACHE_SIZE = 50

##### Sorting users by amount of cache faults of optimal policy


In [97]:
from helpers import belady

reload(belady)

cache_faults = []

for user_id, group in selected_df.groupby("user_id"):
    seq = group["location_id_number"].tolist()
    cache = belady.BeladyCacheAlgorithm()
    faults = cache.get_cache_faults_in_sequence(seq, CACHE_SIZE)
    cache_faults.append(
        (
            user_id,
            len(seq),
            group["location_id_number"].nunique(),
            faults - group["location_id_number"].nunique(),
        )
    )

result_df = pd.DataFrame(
    cache_faults,
    columns=["user_id", "seq_length", "diff_locations", "belady_extra_cache_faults"],
)
result_df = result_df.sort_values(by="belady_extra_cache_faults", ascending=False)

result_df["tenant_rank"] = (
    result_df["belady_extra_cache_faults"]
    .rank(ascending=False, method="first")
    .astype(int)
)
result_df.head(50)

,user_id,seq_length,diff_locations,belady_extra_cache_faults,tenant_rank
194,208,2100,792,164,1
203,2149,2100,646,88,2
149,1863,2100,1295,76,3
150,1864,2100,816,68,4
389,674,1812,641,49,5
142,1791,1393,567,43,6
153,1868,2100,598,35,7
352,62,2100,402,34,8
147,1859,2100,1044,31,9
271,2861,2100,649,31,10


##### Create data for tests, given number of tenants per tests

Users with more faults in optimal cache policy will be used.


In [101]:
tenant_numbers_per_test = [4, 4, 5, 5, 8, 10]

tenant_ranks = np.arange(1, sum(tenant_numbers_per_test) + 1)

selected_df_per_tests = []

for tenant_number in tenant_numbers_per_test:
    tenant_ranks_in_test = np.random.choice(tenant_ranks, tenant_number, replace=False)
    tenant_ranks = np.setdiff1d(tenant_ranks, tenant_ranks_in_test)

    tenant_ids_in_test = result_df[result_df["tenant_rank"].isin(tenant_ranks_in_test)][
        "user_id"
    ].tolist()

    selected_df_per_tests.append(
        selected_df[selected_df["user_id"].isin(tenant_ids_in_test)]
    )

selected_df_per_tests[0].to_csv("../../data/brightkite_4_tenants_1.csv", index=False)
selected_df_per_tests[1].to_csv("../../data/brightkite_4_tenants_2.csv", index=False)
selected_df_per_tests[2].to_csv("../../data/brightkite_5_tenants_1.csv", index=False)
selected_df_per_tests[3].to_csv("../../data/brightkite_5_tenants_2.csv", index=False)
selected_df_per_tests[4].to_csv("../../data/brightkite_8_tenants_1.csv", index=False)
selected_df_per_tests[5].to_csv("../../data/brightkite_10_tenants_1.csv", index=False)